# Parser 12.00 - Short Circuiting Logical AND andOR

The goal of this version is for the parser to recognize the binary infix logical AND and OR operators and for the evaluator to treat them as [short circuiting](https://en.wikipedia.org/wiki/Short-circuit_evaluation). If we can determine what the final result must be after evaluating just the left hand side, the evaluator can skip evaluating the right hand side.

A logical AND operator can produce a *True* result ony if both operands are *True*. If after evaluating the left hand side its value is *False*, the final result must also be *False*. Whatever the result of evaluating the right hand side would be, that final result cannot change. We don't actually have to evaluate the right hand side to know this, so we can safely skip doing so.

Similarly, a logical OR operator can be *True* if either side is *True*. If we find the left hand side evaluates to *True*, we do not need to evaluate the right hand side to determine the final result.

We will use the [C](https://en.wikipedia.org/wiki/C_(programming_language)) symbols **'&&'** and **'||'** to stand for logical AND and OR, respectively. **'&&'** will have a higher precedence than **'||'**.

>Many programming languages, such as [Python](https://en.wikipedia.org/wiki/Python_(programming_language)), [Pascal](https://en.wikipedia.org/wiki/Pascal_(programming_language)) and [Fortran](https://en.wikipedia.org/wiki/Fortran), write them as **'and'** and **'or'**. We could certainly do that as well if we liked.

### What we'll do

Consider a simple logical OR expression such as:

```Python
a = expr1 || expr2
```

If we simply add **'||'**  to the rest of our binary operators and do nothing else, the Reverse Polish will look like:

```Python
a expr1 expr2 B|| B=
```

because **'||'** has a higher precedence than **'=**. That really is all we would have to do. No further changes to the parser would be needed. The evaluator handling of *B||* would also be quite simple:

```Python
def binLogOR(val1, val2):
    pushOperand( 1 if val1 or val2 else 0 )
```

However this form is not suited to short circuiting. When *B||* is executed, both *expr1* and *expr2* are (obviously) already completely evaluated.

>This is not necessarily a bad thing. With only one possible execution path, analysis of program behavior for correctness becomes simpler. Pascal does not support logical short circuiting for precisely this reason.

As we are reckless and daring and actually *want* short circuiting, we have to do something more complicated.

The approach we’re going to take is to split the original binary infix operator into two unary operators in the Reverse Polish. The first follows the left hand expression, the second the right hand expression. So the original expression above will be parsed as:

```Python
a expr1 C||+ expr2 C||- B=
```

> *C* for *conditional*.

The operator *C||+* checks the result of *expr1*. If it is *False*, *C||+* does nothing further, allowing evaluation of *expr2*. *C||-* then checks the result and pushes the final result on the operand stack.

But if the left hand side is *True*, *C||+* pushes *True* on the operand stack and sets flags telling the evaluator to skip everything up to and including *C||-*. In this case *C||'* is never executed and serves only to mark where to stop skipping.

>As an interpreter, our evaluator is going to have to inspect every token in the RPN from *C||+* up to and including *C||-*. A compiler would simply generate a direct jump to just past *C||-* and not bother with all that.

In either case the next operator is *B=*, so the final result is picked up from the operand stack, assigned to **a**, and returned from the evaluator.

Note that we never need the values of both *expr1* and *expr2* at the same time. If we do have to evaluate *expr2*, the mere fact that it was evaluated at all tells us that *expr1* was *False*. This knowledge is what allows both *C||+* and *C||-* to be unary operators.

More complicated expressions are evaluated slightly differently, but the overall behavior is still correct. We’ll detail what happens further on down.

## Libraries

In [ ]:
import glob       # for searching directories
import math
import random     # for 'random()'

import re         # for regular expressions

We import the *random* library to gain access to the *random()* function.

## User output

In [ ]:
visSep = '-------------'             # visual separator

def UIwriteln(this):
    '''write a single line to output'''
    print( f'{this}\n' )
    
def UIwriteSep():
    '''write a visual separator'''
    UIwriteln( visSep )

def UIshow(tag, value):
    '''write a tagged value to output'''
    UIwriteln( f'{tag}: {value}' )

def UIerror(this):
    '''write an error message to output'''
    UIshow( 'Error', this )

# Tracing

In [ ]:
# flags: show trace of processing

showInteract = True          # default for interactive use
showBatch = False            # default for batch use

showTrace = None             # control flag

# Trace Output

def TOshow(mesg, text):
    '''write trace message to output if enabled'''
    if showTrace:
        UIshow( f'{mesg:15s}', text )
        
def TOstring(tag, this):
    
    if showTrace:
        TOshow( tag, ' '.join([str(e) for e in this]) )

# -----------------------
# Parse Tracing
# -----------------------

def PTshowexpr(this):

    TOshow( 'Parse', visSep )
    TOshow( 'Current Expr', this )

def PTshowparse(ok, res, opStk, typStk):

    if ok:
        TOstring( 'Current RPN', res )
        TOstring( 'Operator Stack', opStk )
        TOstring( 'Type Stack', typStk )

def PTshowtoken(this):

    if not this[0] == ' ':
        TOshow( "Found Token", this )

# -----------------------
# Evaluation Tracing
# -----------------------

def ETshowtoken(this):
    
    TOshow( 'Eval', visSep )
    TOshow( 'Current token', this )

def ETshoweval(stk):
    
    TOstring( 'Operand Stack', stk )


# Functions

In [ ]:
def functionErr(nam, msg, val):
    UIerror( f'{nam}(): {msg} value: {val}')
    return ( False, None )

def fncAbs(val):
    '''absolute value of val'''
    return (True, abs(val))

def fncMax(args):
    '''max of two vals'''
    return (True, max(args))

def fncMin(args):
    '''min of two vals'''
    return (True, min(args))

def fncRand():
    '''random decimal'''
    return(True, random.random())

def fncRnd(val):
    '''rounded val'''
    if type(val) is not list:
        return (True, round(val))
    elif type(val[0]) is int:
        return (True, round(val[1], val[0]))
    else:
        return functionErr('ROUND', 'non-integer', val[0])

def fncSgn(val):
    '''sign of val'''
    return (True, 1 if val > 0 else -1 if val < 0 else 0 )

def fncSqt(val):
    '''square root of val'''
    if val >= 0:
        return (True, math.sqrt(val) )
    else:
        return functionErr('SQR', 'negative', val)
    
# known functions

fncDispatch = {
     'ABS': (fncAbs,  1, 1),
     'MAX': (fncMax,  2, None),
     'MIN': (fncMin,  2, None),
  'RANDOM': (fncRand, 0, 0 ),
   'ROUND': (fncRnd,  1, 2),
    'SIGN': (fncSgn,  1, 1),
    'SQRT': (fncSqt,  1, 1)
}


# Parser

In [ ]:
# operands accepted:
# - decimal and hexadecimal floating point literals
# - scalar and array numeric variables
# - numeric functions with zero or more arguments

# operators accepted:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - grouping parentheses
# - logical not, equality, inequality
# - assignment and shortcut assignment
# - prefix and postfix increment and decrement
# - logical short circuit

# errors detected:
# - unrecognized input
# - out of range numeric input
# - malformed expression

# result tuple:
# - (True, [parse])
# - (False, None)

class Parser(object):
    
    VERSIONNUMBER = '12.00'
    
    _FLTMAX =  4294967295                                  # 2**32-1
    _FLTMIN = -4294967296                                  # -(2**32)

    _expMax = {
        'P' : math.log2(_FLTMAX),                          # max base 2 exponent
        'E' : math.log10(_FLTMAX)                          # max base 10 exponent
    }

    _typeChkLst = {                                         # index is left-to-right, check right-to-left
        'an2_': [  None,   'number', 'argsep' ],
         'f2n': ['number', 'fncsym'],
         'n2_': [  None,   'number'],
         'n2n': ['number', 'number'],
         'v2n': ['number', 'numsym'],
        'fn2n': ['number', 'fncsym', 'number'],
        'nn2n': ['number', 'number', 'number'],
        'vn2n': ['number', 'numsym', 'number'],
        'vn2v': ['numsym', 'numsym', 'number']
    }
    
    def __init__(self):
        pass
        
    def doparse(self, this):

        def parseErr(mesg, pos):
            '''report parse error'''
            UIerror(mesg)
            UIwriteln(f'>>> {this}')
            if pos > 0:
                UIwriteln(f'{"^^near here".rjust(pos)}')
            return False

        # initialize

        expr = this                # save to new variable but retain original for error reports
        start = 15                  # tracked so we can report where in an expression an error occurred
        token = None               # anything successfully matched
        ok = wantoperand = True    # flags
        result = []                # rpn expression
        opStk = [ ('EOE', 1) ]     # operator stack
        typStk = []                # type stack
        argStk = []                # function argument count stack
        

        def typeCheck(op, chk):
            '''type check operands'''
            nonlocal argStk
            check = list(self._typeChkLst[chk])       # list() to avoid aliasing
            while len(check) > 1:
                want = check.pop()
                
                # are we checking an argument separator ?
                
                if want == 'argsep':
                    if len(opStk) < 2 or opStk[-2][0] != 'B(':     # within a function call ?
                        return parseErr('Unexpected comma', 0)
                    else:
                        argStk[-1] += 1                            # one more argument
                        result.pop()                               # remove 'F,' from RPN
                        continue
                
                have, rpnPos, errPos = typStk.pop()
                
                # no problem ?
                
                if want == have:
                    continue
                
                # did we want a number ?
                
                if want == 'number':
                    
                    # convert symbols to numbers
                    
                    if have == 'numsym':
                        result.insert( rpnPos, 'U*' )
                                           
                # did we want a variable ?
                
                elif want == 'numsym':
                    return parseErr('Numeric variable expected', errPos)
                    
                # did we want a function name ?
                
                elif want == 'fncsym':
                    
                    if have != 'numsym':
                        return parseErr('Function name expected', errPos)
                    fnc = result[rpnPos-1]
                    if not fnc in fncDispatch:
                        return parseErr(f'Unknown function name: {fnc}', errPos)
                    _, mina, maxa = fncDispatch[fnc]
                    cnt = argStk.pop()
                    if (cnt < mina) or (maxa != None and cnt > maxa):
                        return parseErr(f'Bad argument count: {fnc}', errPos)
                    elif cnt > 1:
                        result.pop()                # remove 'B('
                        result.append(cnt)          # argument count
                        result.append('F()')        # multiple argument function call operator
                    
                # generic everything else
                
                else:
                    return parseErr('Type mismatch', errPos )
                        
            # push result type, RPN operator position, original operator position
                    
            restyp = check.pop()
            if restyp != None:
                typStk.append( (restyp, len(result), errPos) )
            return True
 
        def addOperand(op, typ):
            '''add operand to RPN'''
            result.append( op )
            typStk.append( (typ, len(result), start) )
            
        def addOperator():
            '''add operator to RPN'''            
            op, _, chk = opStk.pop()
            result.append( op )
            return typeCheck( op, chk )
            
        def popGEop(prec):
            '''pop operators of equal or greater precedence'''
            ok = True
            while ok and prec <= opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushLeft(op, prec, chk):
            '''push left associative operator on stack'''
            if not popGEop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popGop(prec):
            '''pop operators of greater precedence'''
            ok = True
            while ok and prec < opStk[-1][1]:
                ok = addOperator()
            return ok
 
        def pushRight(op, prec, chk):
            '''push right associative operator on stack'''
            if not popGop(prec):
                return False
            opStk.append( (op, prec, chk) )
            return True

        def popUntil(op, prec):
            '''clear and check operator stack'''
            if not popGEop(prec):                       # type check failure ?
                return False

            topop = opStk.pop()[0]                      # found match ?
            if op == topop:
                return True
            
            elif op == '(':
                err = 'right parenthesis'
            elif op == '[':
                err = 'right bracket'
            elif topop == '(':
                err = 'left parenthesis'
            elif topop == '[':
                err = 'left bracket'
            else:
                err = 'EOE'
            
            return parseErr( f'Unmatched {err}', start )
   
        # operator dictionaries initialization
        
        _postOps = '[+]{2}|[-]{2}|\(\)'
        
        _postOp = {
            '++': (90, pushLeft, 'v2n'),
            '--': (90, pushLeft, 'v2n'),
            '()': (90, pushLeft, 'f2n')
        }

        _unOps = '!|[+]{1,2}|[-]{1,2}'

        _unOp =  {
            '-':  (80, pushRight, 'n2n'),
            '+':  (80, pushRight, 'n2n'),
            '!':  (80, pushRight, 'n2n'),
            '++': (80, pushRight, 'v2n'),
            '--': (80, pushRight, 'v2n'),
        }

        _binOps = '[-+*/]=?|[=!]?=|\[|\('

        _binOp = {
            '[':  (90, pushLeft,  'vn2v'),
            '(':  (90, pushLeft,  'fn2n'),
            '*':  (70, pushLeft,  'nn2n'),
            '/':  (70, pushLeft,  'nn2n'),
            '+':  (60, pushLeft,  'nn2n'),
            '-':  (60, pushLeft,  'nn2n'),
            '==': (50, pushLeft,  'nn2n'),
            '!=': (50, pushLeft,  'nn2n'),
            '=':  (10, pushRight, 'vn2n'),
            '*=': (10, pushRight, 'vn2n'),
            '/=': (10, pushRight, 'vn2n'),
            '+=': (10, pushRight, 'vn2n'),
            '-=': (10, pushRight, 'vn2n')
        }
        
        _condOps = '&&|[|]{2}'
        
        _condOp = {
            '&&': [(40, pushLeft, 'n2_'), (40, pushLeft, 'n2n')],
            '||': [(30, pushLeft, 'n2_'), (30, pushLeft, 'n2n')]
        }
      
        def convertFloat(fplit, base, capgrp):
            '''convert floating point literal to internal form'''
            
            def rangeErr():
                return parseErr(f'\'{fplit}\' is out of range', start)
            
            # collect the features of interest
                   
            p = re.search(capgrp, fplit.upper())
                            
            lint, lfrc, expbas, expsgn, lexp = p.group(1,2,4,5,6)
            
            # convert integer portion (if any)
 
            uint = 0
            if lint:
                p = re.search('[1-9A-F][0-9A-F]*', lint )
                if p != None:
                    for ch in p.group():
                        digval = '0123456789ABCDEF'.find(ch)
                        if uint <= (self._FLTMAX - digval)/base:
                            uint =  uint * base + digval
                        else:
                            return rangeErr()
                    
            # convert fractional portion (if any)
                    
            ufrc = 0
            if lfrc:
                fbase = 1
                for ch in lfrc:
                    digval = '0123456789ABCDEF'.find(ch)
                    fbase *= base
                    ufrc += digval/fbase
        
            if uint == self._FLTMAX and ufrc != 0:
                return rangeErr()
            
            # value so far
            
            uflt = uint + ufrc
 
            # convert exponent portion (if any)
            
            uexp = 0
            if lexp:
                for ch in lexp:
                    digval = '0123456789'.find(ch)
                    if uexp <= (self._expMax[expbas] - digval)/10:
                        uexp =  uexp * 10 + digval
                    else:
                        return rangeErr()
                    
            # adjust value by exponent (if any)
             
            if uexp:
                power = (2 if expbas == 'P' else 10) ** uexp
                if expsgn == '-':
                    uflt /= power
                elif uflt <= self._FLTMAX/power:
                    uflt *= power
                else:
                    return rangeErr()
                    
            addOperand( uflt, 'numlit' )
            return True

        def startsWith(regex):
            '''test if expression starts with given regular expression'''
            nonlocal expr, start, token

            p = re.match(regex, expr)
            if p == None:
                return False
            else:
                token = p.group()              # what we matched
                start += len(token)            # update to next match position in original string
                expr = expr[len(token):]       # "chop off" what we matched
                PTshowtoken(token)             # trace
                return True

        # top level main loop
              
        while ok and len(expr):

            _ = startsWith('[ ]+')                             # skip leading whitespace

            PTshowexpr(expr)                                   # trace

            # look for operand

            if wantoperand:

                if startsWith('[(]'):
                    '''left parenthesis ?'''
                    opStk.append( ('(', 2) )

                elif startsWith(_unOps):
                    '''right unary?'''
                    prec, assoc, check = _unOp[token]
                    assoc( 'U' + token, prec, check )

                else:

                    wantoperand = False                         # flip
                    
                    if startsWith('0[xX]([0-9a-fA-F]+([.][0-9a-fA-F]*)?|[.][0-9a-fA-F]+)([pP][-+]?[0-9]+)?'):
                        '''unsigned hexadecimal literal ?'''
                        ok = convertFloat(token, 16, '0X([0-9A-F]*)[.]?([0-9A-F]*)(([P])([-+])?([0-9]+))?')

                    elif startsWith('([0-9]+([.][0-9]*)?|[.][0-9]+)([eE][-+]?[0-9]+)?'):
                        '''unsigned decimal literal?'''
                        ok = convertFloat(token, 10, '([0-9]*)[.]?([0-9]*)(([E])([-+])?([0-9]+))?')
                        
                    elif startsWith('[a-zA-Z][_a-zA-Z0-9]*'):
                        '''numeric scalar variable or function name?'''
                        addOperand( token.upper(), 'numsym' )
                        
                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operand', start)

            # look for operator

            else:

                if startsWith('[)\]]'):
                    '''expression terminator?'''
                    ok = popUntil( '(' if token == ')' else '[', 4 )
                        
                    
                elif startsWith(_postOps):
                    '''postfix operator?'''
                    prec, assoc, check = _postOp[token]
                    ok = assoc( 'P' + token, prec, check )
                    if ok and token == '()':
                        argStk.append( 0 )
 
                else:
                    
                    wantoperand = True                          # flip

                    if startsWith(_binOps):
                        '''binary operator?'''
                        prec, assoc, check = _binOp[token]
                        ok = assoc( 'B' + token, prec, check )
                        if ok and (token == '[' or token == '('):
                            opStk.append( (token, 2) )
                            if token == '(':
                                argStk.append( 1 )
                                
                    elif startsWith(_condOps):
                        '''conditional operator?'''
                        op = 'C' + token
                        precl, assocl, checkl = _condOp[token][0]
                        precr, assocr, checkr = _condOp[token][1]
                        ok = assocl( op + '+', precl, checkl) and assocr( op + '-', precr, checkr)

                    elif startsWith(','):
                        '''function argument separator?'''
                        ok = pushLeft( 'F,', 6, 'an2_' )

                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operator', start)

            PTshowparse(ok, result, opStk, typStk )               # trace
 
        if ok and wantoperand:
            ok = parseErr('Unexpected end of expression', start ) # must be in 'wantoperator' state   

        if ok:
            ok = popUntil( 'EOE', 3 )                             # clear operator stack
            
        if ok:
            ok = typeCheck('LoneSymbol', 'n2n')                   # make sure final result is a number

        return (ok, result if ok else None)                       # done
       

### How it works

We'll push two unary operators back-to-back on the operator stack whenever we find one of our binary logical operators. We create a new check for them in the *wantoperator* state.

>If we instead added them to the other binary operators, we'd greatly complicate what to do after finding *any* of them.

We add *_condOps* to be the regular expression that tries to match our logical operators. *_condOp{}* contains two entries for each logical operator we find. The first is for the left hand side and the second for the right hand side. We have to push them both.

>Except in the case where pushing the first results in a parse error. But since Python's logical operator **and** short circuits, pushing the right side operator will be skipped if pushing the left side operator results in an error (returns *False*).

After pushing the left side operator, the expression that appears on the left of the original logical operator is completely in the Reverse Polish and it is now the top operator on the operator stack. Since the right side operator is left associative and has the same precedence, pushing it immediately pops the left side operator into the Reverse Polish and replaces it at the top of the stack. Now the left side operator directly follows the left side expression (which is what we want). The right side operator will stay on the stack until something with an equal or lower precedence pops it off. At that point it will directly follow the right side expression (which is what we want).

And that's almost it.

The new *n2_* "number to nothing" type check for the left hand side does not push a result type. The right side type check produces the original logical operator's result type with an *n2n* check.

# Evaluator

In [ ]:
# operators handled:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - logical negation, equality and inequality
# - variable name de-reference
# - variable assignment and shortcut assignment
# - prefix and postfix increment and decrement
# - numeric array assignment and de-reference
# - function calls with one or more arguments

# errors detected:
# - out of range
# - division by zero
# - invalid function arguments

# return tuple:
# - (True, result)
# - (False, None)

class Evaluator(Parser):
    
    def __init__(self):
        self._symTable = dict()
 
    def doeval(self, rpn):
 
        def inRange(ok, val):
            '''range check test result'''
            if ok:
                stk.append( val )
            else:
                UIerror( 'Evaluation result out of range' )
            return ok
        
        def pushOperand(val):
            '''push operand on stack'''
            stk.append( val )
            return True
        
        def checkSkip(skip, val, down, up):
            '''check and set skip flags'''
            nonlocal skipLevel, downToken, upToken
            if skip:
                skipLevel = 1
                downToken = down
                upToken = up
                stk.append(val)
            return True            
        
        def logLftAnd(val):
            '''left branch of logical AND'''
            return checkSkip(val == 0, 0, 'C&&-', 'C&&+')
        
        def logLftOr(val):
            '''left branch of logical OR'''
            return checkSkip(val != 0, 1, 'C||-', 'C||+')
        
        def logRgt(val):
            '''right branch of logical AND and OR'''
            return pushOperand(1 if val != 0 else 0)
        
        def binAryNdx(ndx, nam):
            '''create array index'''
            return inRange( 0 <= ndx <= self._FLTMAX, f'{nam}_{ndx}' )
                     
        def binVal(val, var):
            '''assign value to variable'''
            self._symTable[var] = val
            return pushOperand(val)
        
        def unVal(var):
            '''variable name value'''
            return pushOperand(self._symTable[var] if var in self._symTable else 0)

#        def unPlu(arg):
#           '''unary plus'''
#           return pushOperand( arg )

        def unNeg(arg):
            '''unary negation'''
            return inRange( arg != self._FLTMIN, -arg )
        
        def unNot(arg):
            '''logical not'''
            return pushOperand( not arg )

        def binAdd(rgt, lft):
            '''binary addition'''
            if lft >= 0:
                return inRange( rgt <= self._FLTMAX - lft, lft+rgt )       
            else:
                return inRange( rgt >= self._FLTMIN - lft, lft+rgt )
 
        def binSub(rgt, lft):
            '''binary subtraction'''
            if lft >= 0:
                return inRange( lft - self._FLTMAX <= rgt, lft-rgt )
            elif rgt >= 0:
                return inRange( lft - self._FLTMIN >= rgt, lft-rgt )
            else:
                return inRange( lft <= self._FLTMAX + rgt, lft-rgt )
 
        def binMul(rgt, lft):
            '''binary multiplication'''
            if abs(lft) <= 1 or abs(rgt) <= 1:
                return pushOperand( lft * rgt )

            elif lft > 0:
                if rgt > 0:
                    return inRange( rgt <= self._FLTMAX / lft, lft * rgt )
                else:
                    return inRange( rgt >= self._FLTMIN / lft, lft * rgt )

            elif rgt > 0:
                return inRange( rgt <= self._FLTMIN / lft, lft * rgt )
  
            else:
                return inRange( rgt >= self._FLTMAX / lft, lft * rgt )
 
        def binDiv(rgt, lft):
            '''binary division'''
            if abs(rgt) >= 1:
                return pushOperand( lft/rgt )
            
            elif rgt > 0:
                if lft > 0 :
                    return inRange( lft <= self._FLTMAX * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMIN * rgt, lft / rgt )
            
            elif rgt < 0:
                if lft > 0:
                    return inRange( lft <= self._FLTMIN * rgt, lft / rgt )
                else:
                    return inRange( lft >= self._FLTMAX * rgt, lft / rgt )

            else:
                UIerror( 'Division by zero' )
                return False
            
        def binEqu(rgt, lft):
            '''logical equality'''
            return pushOperand( lft == rgt )
        
        def binNeq(rgt, lft):
            '''logical inequality'''
            return pushOperand( lft != rgt )
        
        def binShortVal(val, var, op):
            '''shortcut assignment'''
            _ = unVal(var)                           # put the value of the variable on the stack
            if op(val, stk.pop()):                   # perform the arithmetic
                return binVal(stk.pop(), var)        # if successful,  also assign result to variable
            return False
        
        def binAddVal(val, var):
            return binShortVal(val, var, binAdd)
        
        def binSubVal(val, var):
            return binShortVal(val, var, binSub)
        
        def binMulVal(val, var):
            return binShortVal(val, var, binMul)
        
        def binDivVal(val, var):
            return binShortVal(val, var, binDiv)
        
        def unPfxInc(var):
            return binShortVal(1, var, binAdd)
        
        def unPfxDec(var):
            return binShortVal(1, var, binSub)
        
        def unPostFix(val, var):
            '''postfix inc/dec'''
            _ = unVal(var)                             # push current value of variable on stack
            ok = binShortVal(val, var, binAdd)         # update value of variable
            stk.pop()                                  # remove updated value from stack (if error, removes first push)
            return ok
                
        def unPstInc(var):
            return unPostFix(1, var)
            
        def unPstDec(var):
            return unPostFix(-1, var)
        
        def binFnc(val, fnc):
            '''single argument function call'''
            ok, val = fncDispatch[fnc][0](val)
            return pushOperand(val) if ok else False
                
        def multiFnc(cnt):
            '''multiple argument function call'''
            args = []
            while cnt > 0:
                args.append( stk.pop() )
                cnt -= 1
            ok, val = fncDispatch[stk.pop()][0](args)               
            return pushOperand(val) if ok else False
        
        def zeroFnc(name):
            '''zero argument function call'''
            ok, val = fncDispatch[name][0]()
            return pushOperand(val) if ok else False
        
        # initialize
                                
        unDispatch = {
            'U*': unVal,
            'U-': unNeg,
            'U+': pushOperand,
            'U!': unNot,
            'U++': unPfxInc,   'U--': unPfxDec,
            'P++': unPstInc,   'P--': unPstDec,
            'F()': multiFnc,   'P()': zeroFnc,
            'C&&+': logLftAnd, 'C&&-': logRgt,
            'C||+': logLftOr,  'C||-': logRgt
        }
        
        binDispatch = {
            'B+': binAdd, 'B-': binSub,
            'B+=': binAddVal, 'B-=': binSubVal,
            'B*': binMul, 'B/': binDiv,
            'B*=': binMulVal, 'B/=': binDivVal,
            'B==': binEqu, 'B!=': binNeq,
            'B=': binVal,
            'B[': binAryNdx, 'B(': binFnc
        }
  
        skipLevel = 0
        downToken = upToken = None
        stk = []
        ok = True

        # main loop
        
        TOstring('Input', rpn)
        
        for v in rpn:

            ETshowtoken(v)

            if skipLevel > 0:
                TOshow('Skip level', f'{skipLevel}')
                if v == downToken:
                    skipLevel -= 1
                elif v == upToken:
                    skipLevel += 1
            
            elif v in binDispatch:
                ok = binDispatch[v](stk.pop(), stk.pop())
                
            elif v in unDispatch:
                ok = unDispatch[v](stk.pop())
                
            else:
                stk.append( v )

            if not ok:
                return ( False, None )

            ETshoweval( stk )

        return ( True, stk.pop() )


### How it works

We need a way to inform the evaluator that certain portions of an expression should be skipped over without attempting to evaluate them.

For this we introduce the flag variable *skipLevel*. Initialized to zero at the start of evaluation, it can be set to one by either *C&&+* or *C||+*. While *skipLevel* is non-zero no attempt is made to evaluate any token. The evaluator only examines them to determine when to increase or decrease *skipLevel*. When *skipLevel* again becomes zero, skipping will stop. Normal evalution will resume starting with the token after the one which halted skipping.

For example, the expression:

```Python
expr1 || expr2
```

is parsed into RPN as:

```Python
expr1 C||+ expr2 C||-
```

Suppse *expr1* evaluates to zero. In that case *C||+* has no effect, and *expr2* will be evaluated. When *C||-* is executed, it sets the final result.

>It is a curious fact that both *C||-* and *C&&-* can execute using the same function. This is because if this point has been reached, the left side of **&&** must have been non-zero and the left side of **||** zero. Hence if the right side is non-zero, both operators have a *True* result. If it is zero, both operators have a *False* result.

But if *expr1* is non-zero, the final result is set to one and skipping is initiated. While skipping no operator is executed and no operand is pushed on the stack. The evaluator reacts only to *C||+* and *C||-* tokens, raising *skipLevel* for every *C||+* and lowering it for every *C||-*.

For the simple expression above, skipping will stop as soon as *C||-* is reached. What about more complicated expressions?

```Python
expr1 || expr2 || expr3
```

is parsed into RPN as:

```Python
expr1 C||+ expr2 C||- C||+ expr3 C||-
```

Because our two unary operators have the same precedence, the second *C||+* operator pops the first *C||-* operator off the operator stack before being popped off into the Reverse Polish itself by the second *C||-*.

Evaluation still short circuits correctly, but perhaps not obviously. For example, if *expr1* is non-zero, we need to skip all the way from the first *C||+* to the second *C||-* without any further evaluation. This happens in stages. The first *C||+* pushes *True* on the operand stack and sets the skip flag. Skipping stops when the first *C||-* resets *skipLevel* to zero. Normal evaluation resumes starting with the second *C||+*. When executed it finds the *True* placed on the operand stack by the first *C||+*. It pushes back its own *True* and sets the skip flag again. This won't be cleared until reaching the second *C||-* and the end of the entire expression.

Which is what we want.

Suppose *expr1* is instead zero. Then the first *C||+* does nothing and *expr2* is evaluated. The first *C||-* pushes *True* or *False* based on that value. The second *C||+* checks that value. If it is non-zero then the second *C||+* pushes *True* and sets the skip flag. If instead it is zero, the skip flag is not set, *expr3* is evaluated and the final result set by the second *C||-*.

Which is what we want.

The point is that if more than one of the same logical operator appears in an expression, each one after the first checks the result left by the one before it to decide what to do.

In the second example *C||+* and *C||-* appear in alternating sequence. No skip initiated by any *C||+* can extend past the next *C||-*. Even though the next *C||+* can start skipping again, skipping never goes more than one level deep. So why does the main loop check for that possibility? Because an expression like:

```Python
expr1 && (expr2 && expr3)
```

is parsed into RPN as:

```Python
expr1 C&&+ expr2 C&&+ expr3 C&&- C&&-
```

Here our unary operators do not alternate in sequence. During parsing the parentheses prevent the first *C&&-* from being popped by the second *C&&+*. It isn’t popped until the end of the expression is reached.

>At which point the first *C&&-* pushed on the operator stack actually becomes the second *C&&-* in the Reverse Polish.

If *expr1* is *False*, *skipLevel* will become greater than one when the second *C&&+* is reached. The evaluator will skip all the way to the end of the Reverse Polish without executing another operator.

If instead *expr1* is *True*, the final *C&&-* will check – perhaps affirm is a better word, since it won’t change anything – the result of *expr2 && expr3*.

In any case, when the same logical operator is used sequentially all expressions are correctly skipped or evaluated. Short circuiting may cause the evaluator to re-examine a final result in order to keep the short circuit going, but that does not change what the final result is.

One final note: if the final result is the same whether or not skipping happens – short circuiting is, after all, supposed to have the same result as if both sides were evaluated and then the logical operator applied – how do we know it really happens?

One way to know with this particular parser/evaluator is to examine the trace display during interactive evaluation. While skipping tokens will be displayed but nothing will be done with them.

Another way to check that does not require tracing is to place an expression that will cause an evaluation time error in a branch that is supposed to be skipped. Division by zero, say. That will parse, but not evaluate. If the branch is not skipped when it ought to be, we will quickly know about it.

>Short circuiting is actually a two-edged sword. If for some reason a particular logical branch is always skipped during evaluation, any evaluation time errors it contains may never be discovered. Not simply division by zero, but out of bounds array accesses, references to non-existent variables, and so on. If all possible branches are not deliberately checked during testing, it is possible that some future change in the code or the conditions it runs under might cause a previously skipped branch to be evaluated for the first time – with potentially disastrous results.

## Running the parser

In [ ]:
passCnt = failCnt = 0                          # most useful for test input files, but never any harm

myParser = myEvaluator = None                  # where we keep instances of our classes

def startUp(flag):
    '''begin execution'''
    global passCnt, failCnt, showTrace
    global myParser, myEvaluator
    if not myParser:
        myParser = Parser()
    if not myEvaluator:
        myEvaluator = Evaluator()
    UIshow( 'Parser', myParser.VERSIONNUMBER )
    passCnt = failCnt = 0
    showTrace = flag
    
def shutDown():
    '''terminate execution'''
    UIwriteSep()
    UIshow( 'Pass', passCnt )
    UIshow( 'Fail', failCnt )
    
# run parser
        
def parseOne(this):
    '''parse/evaluate one expression'''
    global passCnt, failCnt
    UIwriteSep()
    neg = this[0] == '@'
    if neg:
        this = this[1:]
    UIshow( 'Input', this )
    ok, res = myParser.doparse( this )
    if ok:
        UIshow( 'Final Parse', res )
        ok, res = myEvaluator.doeval( res )
        if ok:
            UIshow( 'Final Eval', res )
        if neg:
            ok = not ok
    if ok:
        passCnt += 1
    else:
        failCnt += 1

## Interactive use

In [ ]:
def parse():
    
    startUp(showInteract)
    while True:
        inp = input( 'Expression: ' )
        UIwriteln( '' )                      # looks better with a blank line here
        if inp.upper()[0] == 'Q':
            break
        elif inp.strip():
            parseOne( inp )
    shutDown()

## Batch processing

In [ ]:
testDir = '..\\ParserTest\\'            # directory holding test input files (empty string if same as notebook directory)

# convert current version number to match test file numbers
# - done this way so we can update only the version number and everything still works

def currNum():
    
    head = myParser.VERSIONNUMBER[:len(myParser.VERSIONNUMBER)-3]
    tail = myParser.VERSIONNUMBER[-2:]
    return f'{head:0>2}{tail}'

# make full path name to test file

def makePath(typ, num):
    return f'{testDir}{typ}{num}.txt'

# run one test

def runTest(this):
    
    UIwriteln(f'Parser {myParser.VERSIONNUMBER} vs {this[-12:-4]}')
    
    myEvaluator._symTable.clear()
    
    with open(this) as f:
        data = f.readlines()
    for line in data:
        test = line.strip()
        if test and test[0] != '#':         # skip blank and comment lines
            parseOne(test)
    
# run a test of current or specified version which should succeed
    
def good(num='curr'):
  
    startUp(showBatch)
    runTest(makePath('pass', currNum() if num == 'curr' else num))
    shutDown()
    
# run a test of current or specified version which should fail

def bad(num='curr'):
    
    startUp(showBatch)
    runTest(makePath('fail', currNum() if num == 'curr' else num))
    shutDown()
    
# run regression test against current and all previous test files

def regress():
            
    UIwriteln('PASS tests')
    
    startUp(showBatch)                       # must create objects before we can access variables inside them 
    currFn = makePath('pass', currNum())
    failed = []
    fnlist = glob.glob(f'{testDir}pass????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = failCnt
            runTest(fn)
            if atstart < failCnt:
                failed.append(fn)               
    shutDown()
    
    UIwriteln('FAIL tests')
    
    startUp(showBatch)
    currFn = makePath('fail',currNum())
    passed = []
    fnlist = glob.glob(f'{testDir}fail????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = passCnt
            runTest(fn)
            if atstart < passCnt:
                passed.append(fn)                
    shutDown()
    
    if not len(failed):
        UIwriteln('All pass tests succeded')
    else:
        UIwriteln('Pass tests which failed')
        for fn in failed:
            UIwriteln(f'  {fn}')
            
    if not len(passed):
        UIwriteln('All fail tests succeded')
    else:
        UIwriteln('Fail tests which passed')
        for fn in passed:
            UIwriteln(f'   {fn}')
              

# Testing the parser

In [ ]:
parse()       # interactive, one expression at a time

In [ ]:
good()        # run current parser against its own pass test. Use good('1234') to run against specific pass test.

In [ ]:
bad()         # run current parser against its own fail test. Use bad('5678') to run against specific fail test.

In [ ]:
regress()     # run parser against all previous tests